# Role: Medical Device Computer
### This notebook acts like an verifier in blockchain. What we do is to fetch the operation from device wallet. And then, we will examine the operation safety to prevent improper operations from hackers. Finally, we will do the action based the operation.

## 1. Instatiate Controller for Verifiers's Agent

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7992
WEBHOOK_BASE = ""
ADMIN_URL = "http://hospital-agent:7991"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic present_proof, printing out the payload. The agent calls to this webhook every time it receives a present proof protocol message from another agent. 


In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

def proof_handler(payload):
    print("Handle present proof")
    print(payload)

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}

agent_controller.register_listeners([connection_listener,proof_listener], defaults=True)



Subscribing too: connections
Subscribing too: present_proof


In [4]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])

ClientConnectorError: Cannot connect to host hospital-agent:7991 ssl:default [Connect call failed ('192.168.0.5', 7991)]

Error during GET /connections: Cannot connect to host hospital-agent:7991 ssl:default [Connect call failed ('192.168.0.5', 7991)]


In [ ]:
#please modidy the corresponding credential id in hospital-issue-device part
credentialId = 'XXXX'

# schema_id = 'EuEtnVakYFyBtGFT1nHYtH:2:SSI PyDentity Tutorial:0.0.1' 

print("Request proof of Fullname and Age range from Identity Holder")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
#TODO - change restriction to schemaId or credentialId

# req_attrs = [
#     {"name": "fullname", "restrictions": [{"schema_id": schema_id}]},
#     {"name": "skill", "restrictions": [{"schema_id": schema_id}]},
# ]
req_attrs = [
    {"name": "fullname", "restrictions": [{"credentialId": credentialId}]},
    {"name": "skill", "restrictions": [{"credentialId": credentialId}]},
]



req_preds = [
    # test zero-knowledge proofs
    {
        "name": "age",
        "p_type": ">=",
        "p_value": 21,
        "restrictions": [{"credentialId": credentialId}],
    }
]

indy_proof_request = {
    "name": "Proof of Completion of PyDentity SSI Tutorial",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

SyntaxError: invalid syntax (2809123619.py, line 2)

### 5b. Send the proof request to Identity Holder

Identity Holder is identified through the connection_id

In [ ]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)


## 6. Verify Proof Presentation

This is checking the signatures on the credentials presented against the credential schema and definition id stored and resolvable on the ledger. It is a bit of a big complicated object, so we show the common pattern for breaking it down, checking it's verified and accessing the data that has been presented.

In [ ]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

### Verifying the State

Once verified through the agent framework api, the state should be 'verified', otherwise the presentation is invalid.

In [ ]:
print(verify['state'])
print(verify['state'] == 'verified')

### Accessing the Revealed Attributes (This is probably talking about how to access the whole data, instead of asking question. Need to be confirmed.)

In [1]:
# print(verify['presentation'])
# print(verify['presentation']['requested_proof'])

for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(val['raw'])


NameError: name 'verify' is not defined

### Accessing self-attested attributes

In [ ]:
for (name, val) in verify['presentation']['requested_proof']['self_attested_attrs'].items():
    print(name)
    ## Slightly different for self attested attrs
    print(val)

# Here is the place you can design the whole medical device computer.

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)

## Well done on completing the PyDentity SSI training series!